In [1]:
%pylab inline
import numpy as np
import scipy as sc
import scipy.ndimage as ndi
import pylab as pl
import matplotlib as mpl
from IPython import display
from itertools import islice
rc("image", cmap="gray")
import dlinputs as dli
import dlmodels as dlm

Populating the interactive namespace from numpy and matplotlib


# Layers


The `dlmodels` library contains some useful layer types:

- Gpu - convert inputs to GPU if necessary
- Img2Seq - convert an image to a sequence by concatenating along the width
- ImgMaxSeq - convert an image to a sequence by max reduction along the width
- ImgSumSeq - convert an image to a sequence by sum reduction along the width
- Lstm1 - 1D LSTM (uni- or bidirectional)
- Lstm1to0 - reduce to final state with 1D LSTM
- Lstm2 - 2D LSTM
- Lstm2to1 - reduce to sequence with LSTM
- Permute - like tensor.permute
- Reorder - reorder (e.g. Reorder("BDL", "BLD"))

Orders are consistently:

- BD - vector inputs
- BDL - sequence inputs (also for Lstm1)
- BDHW - image inputs (also for Lstm2)

# Specs

The `dlmodels` library also contains a small library for compact specifications of deep networks. There is also a TensorFlow version of this library (https://github.com/tmbdev/tfspecs). In addition to a compact notation, `specs` also performs size inference.

In [2]:
with dlm.specops:
    template = (Cbr(64) | Mp(2))**2 | Flat() | Fbr(100) | Fl(10)
model = template.create(17, 1, 28, 28)
model

(17, 1, 28, 28) -> (17L, 10L)
Sequence (
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU ()
  (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (6): ReLU ()
  (7): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (8): Viewer (-1, 3136)
  (9): Linear (3136 -> 100)
  (10): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (11): ReLU ()
  (12): Linear (100 -> 10)
)

# Loadable Models

It is often useful to load models from external files. This can be done with `dlm.loadable.load_model`.